In [2]:
# Dependencies and Setup
!pip install gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# API keys
# OpenWeatherMap API Key
weather_api_key = "42fc9adc90ea2e2c2fdd3e059f13a8de"
# Google API Key
g_key = "AIzaSyAqKJpKDT8joFxSRi-hVSjmLrx1eHgTZc"

In [3]:
#Bring in cities we used in first part
weather_data = pd.read_csv(r"cities_hw.csv")
weather_data

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bethel,90,US,1609809030,74,41.37,-73.41,35.60,4.70
1,1,Fairbanks,40,US,1609808918,82,64.84,-147.72,-11.00,1.36
2,2,Punta Arenas,90,CL,1609808916,62,-53.15,-70.92,51.80,14.99
3,3,Saint Paul Harbor,90,US,1609809031,86,57.79,-152.41,33.80,9.17
4,4,George Town,20,MY,1609808834,88,5.41,100.34,80.60,2.24
...,...,...,...,...,...,...,...,...,...,...
574,574,Port Keats,100,AU,1609809144,80,-14.25,129.55,86.18,9.26
575,575,Pundaguitan,100,PH,1609809144,82,6.37,126.17,81.07,16.84
576,576,Tokur,98,RU,1609809144,84,53.13,132.90,-28.61,0.89
577,577,Nanga Eboko,5,CM,1609809145,95,4.68,12.37,69.12,2.84


In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 70
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 60) & (weather_data["Max Temp"] <= 70)].dropna()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,14,Raja,0,SS,1609809033,29,8.46,25.68,67.75,3.20
291,291,Aswān,0,EG,1609809086,34,24.09,32.91,60.22,8.77
303,303,Oyem,0,GA,1609809089,95,1.60,11.58,67.96,1.95
372,372,Pathein,0,MM,1609809102,76,16.78,94.73,68.43,4.25
374,374,Abu Dhabi,0,AE,1609808943,55,24.47,54.37,62.60,2.24
390,390,Copiapó,0,CL,1609808954,82,-27.37,-70.33,62.60,9.17
425,425,Bago,0,MM,1609808990,94,17.34,96.48,68.00,2.24
432,432,Coyhaique,0,CL,1609808898,34,-45.58,-72.07,66.20,9.17
439,439,Birao,0,CF,1609809115,28,10.28,22.79,68.86,5.97
441,441,Álamos,0,MX,1609809116,65,27.02,-108.93,62.01,4.00


In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df =  pd.DataFrame(hotel_df)
hotel_df.head()

,City,Country,Lat,Lng
14,Raja,SS,8.46,25.68
291,Aswān,EG,24.09,32.91
303,Oyem,GA,1.60,11.58
372,Pathein,MM,16.78,94.73
374,Abu Dhabi,AE,24.47,54.37


In [12]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ''

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,Raja,SS,8.46,25.68,
291,Aswān,EG,24.09,32.91,
303,Oyem,GA,1.60,11.58,
372,Pathein,MM,16.78,94.73,
374,Abu Dhabi,AE,24.47,54.37,
390,Copiapó,CL,-27.37,-70.33,
425,Bago,MM,17.34,96.48,
432,Coyhaique,CL,-45.58,-72.07,
439,Birao,CF,10.28,22.79,
441,Álamos,MX,27.02,-108.93,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [14]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 14: Raja.
Missing field/result... skipping.
------------
Retrieving Results for Index 291: Aswān.
Missing field/result... skipping.
------------
Retrieving Results for Index 303: Oyem.
Missing field/result... skipping.
------------
Retrieving Results for Index 372: Pathein.
Missing field/result... skipping.
------------
Retrieving Results for Index 374: Abu Dhabi.
Missing field/result... skipping.
------------
Retrieving Results for Index 390: Copiapó.
Missing field/result... skipping.
------------
Retrieving Results for Index 425: Bago.
Missing field/result... skipping.
------------
Retrieving Results for Index 432: Coyhaique.
Missing field/result... skipping.
------------
Retrieving Results for Index 439: Birao.
Missing field/result... skipping.
------------
Retrieving Results for Index 441: Álamos.
Missing field/result... skipping.
------------
Retrieving Results for Index 461: Dakar.
Missing field/result... skipping.
------------
Retrieving Results for 

In [19]:
# Display the hotel dataframe
hotel_df.head()
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))